Last Updated: 25 Feb 2021

## Liquidity mining constants

MINING_SECONDS: the period of the liquidity mining programme (in seconds) <br>
TOTAL_REWARD: total number of rewards to be distributed  <br>
EPOCH_SECONDS: the period of an epoch (discretised liquidity mining)

In [1]:
# Generic
MINING_SECONDS = 4*30*86400 # 4 months in seconds 
TOTAL_REWARD = 30e6 # 30mil ROWAN to be distributed
EPOCH_SECONDS = 10*60 # 10min per epoch

# Bundling
constants = {'miningSeconds':MINING_SECONDS,
             'totalReward':TOTAL_REWARD,
             'epochSeconds':EPOCH_SECONDS}

In [2]:
def elementwisesum(listoflists):
    """element-wise summation of lists"""
    return [sum(x) for x in zip(*listoflists)]

In [3]:
# Scenario
"""
LM starts at 00:00
Snapshots were taken every 10 minutes, there were 3 users staking
The following is the state *right before* 02:00
"""

userA = [0, 0, 0, 10e3, 10e3, 10e3, 10e3, 10e3, 10e3, 10e3, 10e3, 10e3]
userB = [10e3, 20e3, 20e3, 30e3, 30e3, 75e3, 80e3, 50e3, 20e3, 20e3, 1e3, 0]
userC = [0, 0, 0, 0, 0, 0, 15e3, 2e3, 0, 5e3, 12e3, 5e3]

list_userSnapshots = [userA, userB, userC]
globalSnapshots = elementwisesum(list_userSnapshots)

## Display live APY

Since Geyser does not provide a higher APY than standard LM, we can just use the APY from a normal LM programme to display the *marginal* APY 

In [4]:
def get_APY(totalStaked, **kwargs):
    """
    Get live APY
    args:
        totalStaked: global total liquidity staked (in ROWAN)
    kwargs:
        miningSeconds: the period of the liquidity mining programme (in seconds)
        totalReward: total number of rewards to be distributed
    returns:
        apy: marginal APY
    """
    miningSeconds, totalReward = kwargs['miningSeconds'], kwargs['totalReward']
    
    if totalStaked > 0: # prevent edge case
        apy = totalReward \
            / totalStaked \
            * 365 * 86400 / miningSeconds \
            * 100
    else:
        apy = 1e9 # show an insanely high APY when no one is providing liquidity
    return apy

In [5]:
# Example
"""
using above-mentioned scenario
all users have the same *marginal* APY
get_APY() is memoryless and user-independent
"""

totalStaked = globalSnapshots[-1]

apy = get_APY(totalStaked=totalStaked, 
              **constants)

print(f'LM APY: {apy:.2f}%')

LM APY: 608333.33%


## Display user accumulated reward

### Normal LM

The user accumulated reward under normal LM

In [6]:
def get_normal_accmulated_reward(userSnapshots, globalSnapshots, **kwargs):
    """
    Get user's accumulated reward under normal liquidity mining
    args:
        userSnapshots: a list of user's liquidity staked at diff snapshots (in ROWAN)
        globalSnapshots: a list of global total liquidity staked at diff snapshots (in ROWAN)
    kwargs:
        miningSeconds: the period of the liquidity mining programme (in seconds)
        totalReward: total number of rewards to be distributed
        epochSeconds: the period of an epoch (in seconds) as we take a snapshot per epoch
    returns:
        userAccReward: user's accumulated reward
    """
    
    assert len(userSnapshots) == len(globalSnapshots), 'Lists have different lengths'
    miningSeconds, totalReward, epochSeconds = kwargs['miningSeconds'], kwargs['totalReward'], kwargs['epochSeconds']
    # total reward distributed per epoch
    totalRewardPerEpoch = totalReward / miningSeconds * epochSeconds
    
    # sum(reward distributed pro-rata at each epoch)
    userAccReward = sum([userStaked / globalStaked * totalRewardPerEpoch for userStaked, globalStaked in zip(userSnapshots, globalSnapshots)])
    return userAccReward

In [7]:
globalAccReward_normal = 0
for userSnapshots, username in zip([userA, userB, userC], ['A', 'B', 'C']):
    
    userAccReward = get_normal_accmulated_reward(userSnapshots=userSnapshots, 
                                                 globalSnapshots=globalSnapshots, 
                                                 **constants)
    globalAccReward_normal += userAccReward
    print(f'User {username} accumulated reward: {userAccReward}')

User A accumulated reward: 4504.63953307675
User B accumulated reward: 14292.157664842467
User C accumulated reward: 2036.5361354141157


In [8]:
print(f'Total accumulated reward: {globalAccReward_normal}')
print(f'Checksum: {TOTAL_REWARD/MINING_SECONDS*EPOCH_SECONDS*len(globalSnapshots)}')

Total accumulated reward: 20833.333333333332
Checksum: 20833.333333333332


### Geyser LM

The user accumulated reward under geyser LM <br>

In [9]:
def get_userSecondsSnapshots(userSnapshots):
    """
    Convert userSnapshots into userSecondsSnapshots for Geyser calculation
    args:
        userSnapshots: a list of user's liquidity staked at diff snapshots (in ROWAN)
    returns:
        userSecondsSnapshots: a list of user's liquidity-epochs (liquidity-seconds) staked at diff snapshots (in ROWAN)
    """
    # initialise
    user_memory = [] 
    userSecondsSnapshots = []

    for i in range(len(userSnapshots)):
        if userSnapshots[i] == 0: # if none staked at snapshot
            user_memory = [] # clear memory
        else: # if some staked at snapshot
            # get the difference between the previous snapshot
            if i == 0:
                diff = userSnapshots[0]
            else:
                diff = userSnapshots[i] - userSnapshots[i-1]

            if diff > 0: # if more tokens are staked
                user_memory.append((i,diff)) # record (index, difference in staked token)
            elif diff < 0: # if some tokens are withdrawn
                deficit = -diff
                while deficit > 0:
                    if user_memory[-1][-1] > deficit: # partial remove
                        user_memory[-1] = (user_memory[-1][0], user_memory[-1][-1]-deficit)
                        deficit = 0
                    else:
                        deficit -= user_memory[-1][-1]
                        user_memory = user_memory[:-1]
        userSecondsSnapshots.append(sum([(i-mem[0]+1)*mem[1] for mem in user_memory]))
    return userSecondsSnapshots

In [10]:
def get_globalSecondsSnapshots(list_userSnapshots):
    """
    Compute globalSecondsSnapshots from a list of userSnapshots
    args:
        list_userSnapshots: a FULL list of userSnapshots (must include all users to capture the global state)
    requires:
        elementwisesum()
    returns:
        globalSecondsSnapshots: a list of global total liquidity-epochs (liquidity-seconds) staked at diff snapshots (in ROWAN)
    """
    list_userSecondsSnapshots = []
    for l in list_userSnapshots:
        list_userSecondsSnapshots.append(get_userSecondsSnapshots(l))
    globalSecondsSnapshots = elementwisesum(list_userSecondsSnapshots)
    return globalSecondsSnapshots

In [11]:
def get_geyser_accmulated_reward(userSnapshots, list_userSnapshots, **kwargs):
    """
    Get user's accumulated reward under geyser liquidity mining
    args:
        userSnapshots: a list of user's liquidity staked at diff snapshots (in ROWAN)
        list_userSnapshots: a FULL list of userSnapshots (must include all users to capture the global state)
    kwargs:
        miningSeconds: the period of the liquidity mining programme (in seconds)
        totalReward: total number of rewards to be distributed
        epochSeconds: the period of an epoch (in seconds) as we take a snapshot per epoch
    requires:
        get_userSecondsSnapshots()
        get_globalSecondsSnapshots()
    returns:
        userAccReward: user's accumulated reward
    """
    
    userAccReward = get_normal_accmulated_reward(userSnapshots=get_userSecondsSnapshots(userSnapshots), 
                                                 globalSnapshots=get_globalSecondsSnapshots(list_userSnapshots), 
                                                 **kwargs)
    return userAccReward

In [12]:
globalAccReward_geyser = 0
for userSnapshots, username in zip([userA, userB, userC], ['A', 'B', 'C']):
    
    userAccReward = get_geyser_accmulated_reward(userSnapshots=userSnapshots, 
                                                 list_userSnapshots=[userA, userB, userC], 
                                                 **constants)
    globalAccReward_geyser += userAccReward
    print(f'User {username} accumulated reward: {userAccReward}')

User A accumulated reward: 4881.305423550728
User B accumulated reward: 15294.48220093532
User C accumulated reward: 657.5457088472862


In [13]:
print(f'Total accumulated reward: {globalAccReward_geyser}')
print(f'Checksum: {TOTAL_REWARD/MINING_SECONDS*EPOCH_SECONDS*len(globalSnapshots)}')

Total accumulated reward: 20833.333333333332
Checksum: 20833.333333333332


## Blind calling live APY & user accumulated reward under Geyser

In [1]:
# Only use functions from utils blindly
from live_data import get_APY, get_geyser_accmulated_reward, elementwisesum

# set constants
constants = {'miningSeconds':4*30*86400,
             'totalReward':30e6,
             'epochSeconds':10*60}

# obtain snapshots from all users
userA = [0, 0, 0, 10e3, 10e3, 10e3, 10e3, 10e3, 10e3, 10e3, 10e3, 10e3]
userB = [10e3, 20e3, 20e3, 30e3, 30e3, 75e3, 80e3, 50e3, 20e3, 20e3, 1e3, 0]
userC = [0, 0, 0, 0, 0, 0, 15e3, 2e3, 0, 5e3, 12e3, 5e3]

list_userSnapshots = [userA, userB, userC]
globalSnapshots = elementwisesum(list_userSnapshots)

In [2]:
apy = get_APY(totalStaked=globalSnapshots[-1], 
              **constants)

print(f'APY: {apy:.2f}%')

APY: 608333.33%


In [3]:
userAccReward = get_geyser_accmulated_reward(userSnapshots=userA, 
                                             list_userSnapshots=[userA, userB, userC], 
                                             **constants)
print(f'User A accumulated reward: {userAccReward}')

User A accumulated reward: 4881.305423550728


In [4]:
userAccReward = get_geyser_accmulated_reward(userSnapshots=userB, 
                                             list_userSnapshots=[userA, userB, userC], 
                                             **constants)
print(f'User B accumulated reward: {userAccReward}')

User B accumulated reward: 15294.48220093532


In [5]:
userAccReward = get_geyser_accmulated_reward(userSnapshots=userC, 
                                             list_userSnapshots=[userA, userB, userC], 
                                             **constants)
print(f'User C accumulated reward: {userAccReward}')

User C accumulated reward: 657.5457088472862
